# 주제분석 2주차 패키지

## Chaper 1. Dataset

### 문제0. 기본세팅

In [1]:
import torch
import torchvision
from torchvision import transforms
import pathlib
import sklearn
import numpy as np
import PIL

### 문제1. 시드설정

In [2]:
import random
np.random.seed(17)
seed = np.random.seed(17)

### 문제2. dog, cat 변수를 만들어 폴더에 들어있는 경로들을 리스트 형태로 저장

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
from pathlib import Path
cat_dir = pathlib.Path('/content/gdrive/My Drive/training_set/cats')
dog_dir = pathlib.Path('/content/gdrive/My Drive/training_set/dogs')

In [7]:
cat = []
for i in cat_dir.iterdir():
    cat.append(i)
    
dog = []
for i in dog_dir.iterdir():
    dog.append(i)

### 문제3. 저장한 리스트를 8:2로 나누고 train/test로 구분

In [8]:
from sklearn.model_selection import train_test_split

cat_train, cat_val = train_test_split(cat, test_size = 0.2, shuffle = True, random_state = seed)
dog_train, dog_val = train_test_split(dog, test_size = 0.2, shuffle = True, random_state = seed)

In [9]:
train = cat_train + dog_train
val = cat_val + dog_val

### 문제4. torchvision.transforms를 이용하여 이미지 데이터 변화

### 4-1. train 데이터셋 변화

In [10]:
train_transforms = transforms.Compose([
    transforms.Resize((128,128)), #사이즈 조정
    transforms.RandomHorizontalFlip(), #이미지를 수직변환할 수 있게 하여 방향변화에 robust한 형태로 만들기
    transforms.ToTensor() #PIL 형태의 이미지나 ndarray 를 PyTorch 가 이해할 수 있는 tensor 자료형으로 바꾸어 주는 역할
])

### 4-2. val 데이터셋 변화

In [11]:
val_transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

### 문제5. Dataset class 만들기

In [12]:
from torch.utils.data import Dataset
#데이터의 양이 방대할 경우, 데이터를 한번에 불러오기 어렵기 때문에, 하나씩 불러서 쓰는 방식을 택하기 위해 Dcustomized된 ataset을 사용함
#미니배치 학습, 데이터 셔플, 병렬 처리까지 간단하게 수행

class Dataset(Dataset) :
    
    def __init__(self, data, transforms): #생성될 때 한번만 실행됨
        self.data = data
        self.transforms = transforms
        
    def __len__(self): #데이터의 개수 
        return len(self.data)
    
    def __getitem__(self, index): #인덱스값을 기준으로 데이터셋에서 sample을 가져오고, 이를 변환하는 과정을 거침
        image_path = self.data[index]
        image = PIL.Image.open(image_path)
        image_transformed = self.transforms(image)
        
        if "cat" in str(self.data[index]) :
            label = 0
        else :
            label = 1
            
        return image_transformed, label

### 6. 만든 Dataset class를 이용하여 train, val 데이터를 Dataset 형식으로 바꾸기

In [13]:
train_dataset = Dataset(data = train, transforms = train_transforms)
val_dataset = Dataset(data = val, transforms = val_transforms)

In [14]:
print(len(train_dataset), len(val_dataset))

6404 1601


In [15]:
train_dataset[0]

(tensor([[[0.6392, 0.6353, 0.6275,  ..., 0.7412, 0.7412, 0.7490],
          [0.6392, 0.6392, 0.6235,  ..., 0.7294, 0.7294, 0.7373],
          [0.6353, 0.6314, 0.6275,  ..., 0.7137, 0.7176, 0.7216],
          ...,
          [0.6314, 0.6078, 0.6000,  ..., 0.6510, 0.5608, 0.5294],
          [0.6196, 0.6078, 0.6039,  ..., 0.7020, 0.6000, 0.5608],
          [0.6039, 0.5961, 0.5922,  ..., 0.7333, 0.5922, 0.5490]],
 
         [[0.5804, 0.5765, 0.5686,  ..., 0.7059, 0.7059, 0.7137],
          [0.5804, 0.5804, 0.5647,  ..., 0.6980, 0.6980, 0.7059],
          [0.5765, 0.5725, 0.5686,  ..., 0.6824, 0.6863, 0.6902],
          ...,
          [0.6039, 0.5804, 0.5725,  ..., 0.6235, 0.5373, 0.5059],
          [0.5922, 0.5804, 0.5765,  ..., 0.6745, 0.5725, 0.5333],
          [0.5765, 0.5686, 0.5647,  ..., 0.7059, 0.5647, 0.5216]],
 
         [[0.5608, 0.5569, 0.5490,  ..., 0.7098, 0.7098, 0.7137],
          [0.5608, 0.5608, 0.5451,  ..., 0.6941, 0.6941, 0.6980],
          [0.5569, 0.5529, 0.5490,  ...,

In [16]:
val_dataset[0]

(tensor([[[0.6471, 0.6314, 0.5020,  ..., 0.7098, 0.8039, 0.8588],
          [0.6510, 0.6431, 0.5569,  ..., 0.6588, 0.6902, 0.7373],
          [0.6314, 0.6196, 0.5804,  ..., 0.6902, 0.7490, 0.7294],
          ...,
          [0.7725, 0.7608, 0.7647,  ..., 0.7490, 0.7451, 0.7412],
          [0.7843, 0.7647, 0.7804,  ..., 0.7529, 0.7255, 0.7255],
          [0.7765, 0.7608, 0.7647,  ..., 0.7529, 0.7412, 0.7373]],
 
         [[0.6235, 0.6196, 0.4627,  ..., 0.6196, 0.7608, 0.8275],
          [0.6235, 0.6275, 0.5216,  ..., 0.5647, 0.6157, 0.6745],
          [0.6039, 0.6078, 0.5569,  ..., 0.6118, 0.6588, 0.6392],
          ...,
          [0.7451, 0.7294, 0.7333,  ..., 0.7176, 0.7137, 0.7098],
          [0.7529, 0.7333, 0.7490,  ..., 0.7216, 0.6941, 0.6941],
          [0.7451, 0.7294, 0.7333,  ..., 0.7176, 0.7098, 0.7059]],
 
         [[0.5961, 0.5216, 0.3098,  ..., 0.4392, 0.5529, 0.6314],
          [0.5922, 0.5373, 0.3882,  ..., 0.4118, 0.4275, 0.4667],
          [0.5647, 0.5255, 0.4549,  ...,

### 7. Dataset을 Dataloader로 만들기

In [17]:
from torch.utils.data import DataLoader
#batch형태로 만들어서 우리가 실제로 학습할 때 이용할 수 있게 형태를 만들어주는 라이브러리 : DataLoader

train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 64, shuffle = True)

In [18]:
train_dataloader

In [19]:
val_dataloader

## 2. Modelling

### 1. CNN 모델을 class 형태로 만들어주기

In [20]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() ## 기반 class를 초기화하고, 해당 class 속성을 subclass가 스스로 받아오도록 함!
        
        ### Conv2d -> BathNorm -> ReLU -> Maxpool
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3), ###in_channels) 흑백이면 1, RGB면 3
            nn.BatchNorm2d(16),
            ### batch정규화 : 각 배치 단위 별로 데이터가 다양한 분포를 가지더라도 각 배치별로 평균과 분산을 이용해 정규화
            ### batch parameter의 기준은 input size
            nn.ReLU(),
            ### 양수면 자기자신, 음수면 0을 반환 / 갈수록 기울기가 소실되는 시그모이드 개선(0~1사이의 값만 내보냄)
            ### 가중치 업데이트 속도, 연산 속도가 매우 빠르고(함수가 간단), 기울기 소실(vanishing Gradient)문제가 발생하지 않음
            ### feature가 발현될 가능성을 비선형적 그래프로 나타내주는 작업이 필요(non-linearity 부여)
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            ### pooling은 파라미터를 줄여 overfitting을 방지하고, 속도를 높이기 위한 방법
            ### convolution layer를 resize하여 새로운 layer를 얻는 것, 특징을 뽑아내는 작업
        )    
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc = nn.Linear(1024,2) #평탄화 작업, Convolution/Pooling 프로세스의 결과를 취하여 다음 결과와 이어질 수 있게 연결하는 역할
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        
        x = x.reshape(-1,1024)
        
        x = self.fc(x) ### fc layer를 거친 값으로 반환
        
        return x

### 2. 그래픽카드로 장치 설정 후, 모델 그래픽 카드에 할당

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

### 3. optimizer와 loss function 정의(모델 학습을 위해 최적화, 손실 함수가 필요함!)

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #learniing rate 설정
### Adam : 모멘텀 최적화 + RMSProp / 모멘텀 최적화처럼 Gradient Descent을 따르고, RMSProp처럼 그레디언트 제곱의 지수 감소 평균을 따름

loss_function = nn.CrossEntropyLoss()
### softmax(classification score을 0~1 사이로 정규화, 모든 합이 1) + crossentropy
### 정답 class에 해당하는 값에 대해서만 log 합 진행

### 4. 2중 for문을 이용한 모델 학습, iteration 100번 지날 때마다 loss 값 출력

In [23]:
num_epoch = 30

for epoch in range(num_epoch) :
  ### enumerate : index와 원소로 이루어진 tuple 생성 (index와 원소에 동시 접근해서 loop를 돌릴 수 있음)
  for index, data in enumerate(train_dataloader) :
    input, label = data
    input, label = input.to(device), label.to(device)
    
    optimizer.zero_grad()
    output = model.forward(input)
    
    loss = loss_function(output, label)
    loss.backward()
    optimizer.step()
    
    if index % 100 == 99:
      print(f'Epoch : {epoch + 1} loss: {loss.item() : .3f}')
print('Finish Training')

Epoch : 1 loss:  0.414
Epoch : 2 loss:  0.467
Epoch : 3 loss:  0.306
Epoch : 4 loss:  0.249
Epoch : 5 loss:  0.287
Epoch : 6 loss:  0.260
Epoch : 7 loss:  0.296
Epoch : 8 loss:  0.210
Epoch : 9 loss:  0.196
Epoch : 10 loss:  0.073
Epoch : 11 loss:  0.132
Epoch : 12 loss:  0.120
Epoch : 13 loss:  0.102
Epoch : 14 loss:  0.066
Epoch : 15 loss:  0.159
Epoch : 16 loss:  0.061
Epoch : 17 loss:  0.018
Epoch : 18 loss:  0.012
Epoch : 19 loss:  0.011
Epoch : 20 loss:  0.025
Epoch : 21 loss:  0.035
Epoch : 22 loss:  0.018
Epoch : 23 loss:  0.054
Epoch : 24 loss:  0.075
Epoch : 25 loss:  0.023
Epoch : 26 loss:  0.011
Epoch : 27 loss:  0.004
Epoch : 28 loss:  0.001
Epoch : 29 loss:  0.015
Epoch : 30 loss:  0.006
Finish Training


### 5. 평가모드로 변환

In [24]:
model.eval() ##해당 모델의 모든 layer가 eval모드에 적용됨, Dropout이나 Batchnorm 비활성화

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

### 6. Train Accuracy 구하기

In [27]:
correct = 0
total = 0

with torch.no_grad():
    for input,label in train_dataloader:
        x = input.to(device)
        y= label.to(device)

        output = model.forward(x)
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()
    
    # 정확도 도출
    print("Accuracy of Train Data: {}%".format(100*correct/total))

Accuracy of Train Data: 100.0%


### 7. Validation Accuracy 구하기

In [29]:
correct = 0
total = 0
mis_predicted_value = []

with torch.no_grad():
    for input,label in val_dataloader:
        x = input.to(device)
        y= label.to(device)

        output = model.forward(x)
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()

        # mispredict값 관련
        for mispredict in range(len(output_index != label)) :
            if mispredict == True :
                mis_predicted_value.append([data[0][mispredict], data[1][mispredict]])

    # 정확도 도출
    print("Accuracy of Validation Data: {}%".format(100*correct/total))

Accuracy of Validation Data: 89.06932830810547%


### 8. 잘못 예측한 데이터 살펴보기

In [30]:
mis_predicted_value[0][0]

tensor([[[0.5098, 0.5333, 0.5059,  ..., 0.5137, 0.4549, 0.4549],
         [0.4902, 0.5216, 0.4784,  ..., 0.5451, 0.4824, 0.4078],
         [0.5373, 0.5176, 0.5255,  ..., 0.5922, 0.4431, 0.2745],
         ...,
         [0.4784, 0.5294, 0.4706,  ..., 0.4431, 0.4235, 0.3490],
         [0.4902, 0.4902, 0.3882,  ..., 0.4784, 0.4235, 0.5020],
         [0.5804, 0.5451, 0.3412,  ..., 0.4667, 0.5412, 0.6118]],

        [[0.5686, 0.5922, 0.5608,  ..., 0.4510, 0.3725, 0.3804],
         [0.5373, 0.5686, 0.5294,  ..., 0.4980, 0.4196, 0.3686],
         [0.5686, 0.5569, 0.5686,  ..., 0.5647, 0.3961, 0.2314],
         ...,
         [0.5176, 0.4745, 0.4039,  ..., 0.4824, 0.4941, 0.4039],
         [0.4980, 0.4863, 0.3961,  ..., 0.5137, 0.4941, 0.5412],
         [0.5686, 0.5725, 0.3961,  ..., 0.5059, 0.5961, 0.6314]],

        [[0.3529, 0.3804, 0.3569,  ..., 0.4510, 0.3725, 0.3490],
         [0.3647, 0.3843, 0.3294,  ..., 0.4824, 0.4157, 0.3647],
         [0.4314, 0.3961, 0.3765,  ..., 0.5176, 0.3765, 0.

### 9. Torch vision의 models에서 pretrained된 resnet34 불러오기

In [33]:
from torchvision import models

resnet34 = models.resnet34(pretrained = True)
print(resnet34)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 10. 불러온 모델의 fc layer를 문제에 맞게 바꾸어주기

In [34]:
### pretrained된 딥러닝 모델을 binary classification에 사용하기

nr_filters = resnet34.fc.in_features  # number of input features of last layer
resnet34.fc = nn.Linear(nr_filters, 2) # binary classification -> 2

resnet34.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 11. Fine-tuning과 transfer learning의 차이

transfer learning 방법 중 하나가 fine tuning이다.

- transfer learning : pretrained 모델을 가져와 분석자가 가진 데이터셋으로 재학습시키는 과정.
- fine tuning : pretrained 모델을 가져와 분석자가 가진 데이터셋으로 재학습 시키는 것에 더하여, 분석 목적에 맞게 모델 architecture를 변형한다! ex) 범주분류의 개수를 줄이거나, 출력층을 목적에 맞게 변형시키는 것!

### 12. 불러온 모델을 train data로 학습시키기! epoch은 10회

In [35]:
optimizer = torch.optim.Adam(resnet34.parameters(), lr = 0.001) #learniing rate 설정
### Adam : 모멘텀 최적화 + RMSProp / 모멘텀 최적화처럼 Gradient Descent을 따르고, RMSProp처럼 그레디언트 제곱의 지수 감소 평균을 따름

loss_function = nn.CrossEntropyLoss()
### softmax(classification score을 0~1 사이로 정규화, 모든 합이 1) + crossentropy
### 정답 class에 해당하는 값에 대해서만 log 합 진

num_epoch = 10

for epoch in range(num_epoch) :
  ### enumerate : index와 원소로 이루어진 tuple 생성 (index와 원소에 동시 접근해서 loop를 돌릴 수 있음)
  for index, data in enumerate(train_dataloader) :
    input, label = data
    input, label = input.to(device), label.to(device)
    
    optimizer.zero_grad()
    output = model.forward(input)
    
    loss = loss_function(output, label)
    loss.backward()
    optimizer.step()
    
    if index % 100 == 99:
      print(f'Epoch : {epoch + 1} loss: {loss.item() : .3f}')
print('Finish Training')

Epoch : 1 loss:  0.003
Epoch : 2 loss:  0.001
Epoch : 3 loss:  0.001
Epoch : 4 loss:  0.003
Epoch : 5 loss:  0.002
Epoch : 6 loss:  0.003
Epoch : 7 loss:  0.003
Epoch : 8 loss:  0.002
Epoch : 9 loss:  0.003
Epoch : 10 loss:  0.001
Finish Training


### 13. 모델을 평가모델로 바꾼 후, train accuracy와 validation accuracy 구하기

In [36]:
resnet34.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
### train accuracy
correct = 0
total = 0

with torch.no_grad():
    for input,label in train_dataloader:
        x = input.to(device)
        y= label.to(device)

        output = resnet34.forward(x)
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()
    
    # 정확도 도출
    print("Accuracy of Train Data: {}%".format(100*correct/total))

Accuracy of Train Data: 49.96876907348633%


In [41]:
### validation accuracy
correct = 0
total = 0

with torch.no_grad():
    for input,label in val_dataloader:
        x = input.to(device)
        y= label.to(device)

        output = resnet34.forward(x)
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()

    # 정확도 도출
    print("Accuracy of Validation Data: {}%".format(100*correct/total))

Accuracy of Validation Data: 49.96876907348633%
